<table>
    <tr>
        <td><img src='https://coastalrisk.live/wp-content/uploads/2018/05/cera_50x50.png' alt='Image' width='50' height=50'></td>
        <td><h1 align="left"><font color='green'> Introduction to CERA Matplotlib Contouring for ADCIRC NetCDF Data </font></h1></td>
    </tr>
</table>


In [15]:
# This script uses the python library matplotlib (http://matplotlib.org/) to create contours from a single ADCIRC netcdf file.
# If multiple time steps are given in the ADCIRC netcdf file, the first time step will be extracted.

# Copyright (C): Carola Kaiser 2014-2024, Louisiana State University. With special thanks to from the Matplotlib developer 
# team for helping us create clean geometries.
# This script is part of the Coastal Emergency Risks Assessment (CERA), a real-time visualization system for storm surge guidance.
# See https://cera.coastalrisk.live. This CERA script is Open Source software; distributed under the MIT License.

# Script specifics and bugfixes:
# Matplotlib 'tricontourf' expects a data array, but does not support masked arrays. If a masked array will be passed, it will be ignored.
# The triangulation should only contain triangles with valid data at all three vertices. The solution is to either remove invalid triangles from
# your 'element' array before creating the triangulation, or set a mask on the triangulation once it has been created.
# The created contours will contain one or more polygon exteriors and zero or more interiors. They can be in any order (an exterior is not
# necessarily followed by its interiors). This has to be explicitly tested in your own script.

# Table of Contents

- [Part 1: Introduction](#part-1-introduction)
  - [1.1 - Software Installation](#11---software-installation)
  - [1.2 - Downloading the netCDF data](#12-downloading-the-netcdf-data.)

- [Part 2: Data Import](#part-2-data-import)
  - [2.1 - Importing Necessary libraries](#21---importing-necessary-libraries)
  - [2.2 - Opening a NetCDF file](#22---opening-a-netcdf-file)
  - [2.3 - Extracting and Validating Grid Variables from NetCDF File](#23---extracting-and-validating-grid-variables-from-netcdf-file)
  - [2.4 - Extracting and Validating Attribute Data from NetCDF File](#24---extracting-and-validating-attribute-data-from-netcdf-file)

- [Part 3: Defining essential functions](#part-3-defining-essential-functions)
  - [3.1 - Calculating signed area of a polygon](#31---calculating-signed-area-of-a-polygon)
  - [3.2 - Classifying polygons](#32---classifying-polygons)
  - [3.3 - Checking Points Within a Polygon](#33---checking-points-within-a-polygon)
  - [3.4 - Reversing the order of elements](#34---reversing-the-order-of-elements)
  - [3.5 - Geometry extraction](#35---geometry-extraction)
  - [3.6 - Interactive plotting](#36---interactive-plotting)

>Disclaimer: If you are running through google Colab please use the table of contents from the top left corner.

# Part 1: Introduction
CERA provides real-time visualization and guidance for storm surge events in the Northern Gulf and Atlantic Coast regions.The purpose of this script is to generate contours from ADCIRC NetCDF files using the Matplotlib library. These contours can then be utilized to produce various GIS file formats and map plots, essential for storm surge analysis and emergency management.

This script significantly enhances matplotlib's 'tricontourf' function by accommodating masked arrays, a feature not supported by the original function. It achieves this by either eliminating invalid triangles prior to triangulation or applying a mask post-creation. Moreover, the script streamlines the process of working with contours, which can comprise multiple exteriors and interiors in any sequence, by autonomously managing these intricacies. 

These improvements are not just technical details, they are at the heart of the [CERA](https://cera.coastalrisk.live/) website's functionality. In this Jupyter Notebook tutorial, we'll explore these nuances of using Matplotlib's 'tricontourf' function, thereby illuminating the inner workings of CERA, a critical tool for coastal risk assessment.

## 1.1 - Software Installation


### Installing Jupyter Notebook

Here are listed the various options for installing jupyter notebook in your system. Please choose according to your system requirments and personal preference. 

- [Installing the classic Jupyter Notebook interface](https://docs.jupyter.org/en/latest/install/notebook-classic.html)
- [Installing the JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)



#### Our prefered way of working is using [Viusal Studio Code](https://code.visualstudio.com/download)
The general instruction are [here](https://code.visualstudio.com/docs/datascience/jupyter-notebooks) using the extension provided by Microsoft [Jupyter](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter).

Pick the prefered package manager / environment manager (conda or venv)\
Open any jupyter notebook file with the extentssion .ipynb in vscode.\
From the top right corner pick as a kernel for this specific jupyter notebook file the enviroment created in the previous step.

### Standalone Installation

For running the tutorial Python script examples using a command-line or terminal, the following software should be installed:
- optparse-pretty
- netCDF4
- matplotlib
- shapely
- numpy

Disclaimer: Please use the cera_contour_matplotlib.py to run standalone python (not notebook)

### Install the necessary libraries for Standalone python

```
pip install optparse-pretty
pip install netCDF4
pip install matplotlib
pip install shapely
pip install numpy
```

### Install the necessary libraries Jupiter Notebook

#### Explaining the libraries

- `netCDF4` is a Python interface to the netCDF C library. netCDF (Network Common Data Form) is a set of software libraries and self-describing, machine-independent data formats that support the creation, access, and sharing of array-oriented scientific data.
- `matplotlib` is a plotting library for Python. It provides an object-oriented API for embedding plots into applications using general-purpose GUI toolkits like Tkinter, wxPython, Qt, or GTK. Matplotlib is also a popular library for creating static, animated, and interactive visualizations in Python.
- `shapely` is a Python package for set-theoretic analysis and manipulation of planar features using functions from the GEOS library (which is the engine of PostGIS).
- `numpy` is a Python library that provides support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
- `ipywidgets` are interactive HTML widgets for Jupyter notebooks and the IPython kernel. Widgets in ipywidgets are objects that represent a control, like a slider, a text box, a button, etc. They can be used to build interactive GUIs for your notebooks, handle user input, and render interactive output. Also, they can also be used to synchronize stateful and stateless information between Python and JavaScript.



In [16]:
!pip install netCDF4
!pip install matplotlib
!pip install shapely
!pip install numpy
!pip install ipywidgets

## 1.2 Downloading the netCDF data.

In [17]:
!wget https://cloud.cera.lsu.edu/s/frcZtNgANgdgCXP/download/maxele_contouring.63.nc

--2024-04-08 22:53:34--  https://cloud.cera.lsu.edu/s/frcZtNgANgdgCXP/download/maxele_contouring.63.nc
Resolving cloud.cera.lsu.edu (cloud.cera.lsu.edu)... 130.39.22.133
Connecting to cloud.cera.lsu.edu (cloud.cera.lsu.edu)|130.39.22.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8418147 (8,0M) [application/octet-stream]
Saving to: ‘maxele_contouring.63.nc’

maxele_contouring.6 100%[===================>]   8,03M  5,86MB/s    in 1,4s    

2024-04-08 22:53:36 (5,86 MB/s) - ‘maxele_contouring.63.nc’ saved [8418147/8418147]



# Part 2: Data Import

## 2.1 - Importing Necessary libraries


In [18]:
# importing libraries
import os, sys
import netCDF4
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from matplotlib import path
from shapely.geometry import geo, Polygon, Point, MultiPolygon
import numpy
from operator import itemgetter
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

## 2.2 - Opening a NetCDF file 

```python
gridfile = "maxele_contouring.63.nc"
```
- This line of code assigns the string "maxele_contouring.63.nc" to the variable `gridfile`. This string is presumably the name of the netCDF file that contains the grid data.

```python
gridvars = netCDF4.Dataset(gridfile).variables
```
- This line uses the `netCDF4.Dataset` function to open the netCDF file specified by `gridfile`. The `Dataset` function returns a `Dataset` object that represents the netCDF file. The `.variables` attribute of a `Dataset` object is a dictionary that contains all the variables in the netCDF file. This dictionary is assigned to the `gridvars` variable.

In [19]:
# Read x,y from the grid file or the input file if grid is not given
gridfile = "maxele_contouring.63.nc"
gridvars = netCDF4.Dataset(gridfile).variables

## 2.3 - Extracting and Validating Grid Variables from NetCDF File

Let's read 'x', 'y', and 'element' (or 'lon', 'lat', and 'ele' if the former are not present) data from the netCDF file stored in `gridvars`, adjusting the element indexing from 1-based to 0-based. 

We can also check if any of these data arrays are `None`, and if so, print an error message and exit the program.

In [20]:
# Get variable names for x, y depending on grid
if 'x' in gridvars:
    var_x = 'x'
    var_y = 'y'
    var_element = 'element'
else:
    var_x = 'lon'
    var_y = 'lat'
    var_element = 'ele'

# Read x, y, and elements from the grid file
x = gridvars[var_x][:]
y = gridvars[var_y][:]

elems = gridvars[var_element][:, :] - 1  # Move to 0-indexing by subtracting 1, elements indexing starts with '1' in netcdf file

if x is None or y is None or elems is None:
    print("*** ERROR *** No 'x (lon)', 'y (lat)', or 'element (ele)' data array given in file '%s'" % gridfile)
    sys.exit(-1)

The `gridvars.keys()` method returns a view object that displays a list of all keys in the `gridvars` dictionary.

`gridvars` is expected to be a dictionary where each key-value pair represents a variable from a netCDF dataset. The keys are the variable names, and the values are the corresponding Variable objects. 

`print(gridvars.keys())` is printing the names of all the variables in the netCDF dataset. This could be useful for understanding what data is available in the dataset, for debugging, or for logging.

In [21]:
print(gridvars)
print(gridvars.keys())

{'time': <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: model time
    standard_name: time
    units: seconds since 2014-04-07 00:00:00
    base_date: 2014-04-07 00:00:00
unlimited dimensions: time
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'x': <class 'netCDF4._netCDF4.Variable'>
float64 x(node)
    long_name: longitude
    standard_name: longitude
    units: degrees_east
    positive: east
unlimited dimensions: 
current shape = (295328,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'y': <class 'netCDF4._netCDF4.Variable'>
float64 y(node)
    long_name: latitude
    standard_name: latitude
    units: degrees_north
    positive: north
unlimited dimensions: 
current shape = (295328,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'element': <class 'netCDF4._netCDF4.Variable'>
int32 element(nele, nvertex)
    long_name: element
    standard_name: face_node_connectivity
    start_index: 1
    un

## 2.4 - Extracting and Validating Attribute Data from NetCDF File

Reading the 'zeta_max' data array from a netCDF file stored in `gridvars`. If the data array is `None` or contains multiple time steps, print an appropriate message and either exits the program or extract the first time step, respectively.

In [22]:
# Read the 'attribute name' data array from the input file

attrname = "zeta_max"

data = gridvars[attrname][:]

if data is None:
    print("*** ERROR *** No '%s' data array given in file '%s'" % (attrname, infile))
    sys.exit(-1)

# Timesteps layer
if len(data.shape) > 1:
    print("*** The data file contains multiple time steps. The first time step (0) will be extracted.")
    data = data[0]

# Part 3: Defining essential functions

## 3.1 - Calculating signed area of a polygon

Our next step is creating a usefull function that calculates and returns the signed area of a polygon, represented by the input `ring`, by using the cross product method and numpy operations for efficient computation.

In [23]:
def signed_area(ring):
    v2 = numpy.roll(ring, -1, axis=0)
    return numpy.cross(ring, v2).sum() / 2.0

The first line inside the function, `v2 = numpy.roll(ring, -1, axis=0)`, uses the `numpy.roll` function to shift the elements in the `ring` array along the first axis (axis=0). The shift is by -1, which means each element is moved one place towards the beginning of the array. The last element is moved to the first place. This operation effectively gives us a new array where each element is the next point in the polygon relative to the corresponding element in the original `ring` array.

The second line, `return numpy.cross(ring, v2).sum() / 2.0`, calculates the signed area of the polygon. The `numpy.cross` function is used to calculate the cross product of the original points and their "next" points. This gives us the signed area of the parallelogram formed by each point and its next point. The cross product is positive if the points are in counterclockwise order and negative if they are in clockwise order. 


## 3.2 - Classifying polygons

The function `classify_polygons(polys)` is designed to classify a list of polygons into two categories: exterior and interior polygons. 

The parameter `polys` is expected to be a list of polygons, where each polygon is represented as a 2D numpy array of points.

In [24]:
def classify_polygons(polys):
    exteriors = []
    interiors = []
    for p in polys:
        if signed_area(p) >= 0:
            exteriors.append(p)
        else:
            interiors.append(p)

    return exteriors, interiors

Two empty lists, `exteriors` and `interiors`, are initialized at the start of the function. These lists will hold the exterior and interior polygons, respectively.

The function then iterates over each polygon `p` in the `polys` list. For each polygon, it calculates the signed area using the `signed_area(p)` function. The signed area is a measure that is positive for polygons oriented counterclockwise and negative for polygons oriented clockwise.

If the signed area of a polygon is greater than or equal to 0, it means the polygon is oriented counterclockwise and is considered an exterior polygon. In this case, the polygon `p` is appended to the `exteriors` list using the `append()` method.

If the signed area of a polygon is less than 0, it means the polygon is oriented clockwise and is considered an interior polygon. In this case, the polygon `p` is appended to the `interiors` list.

Finally, the function returns a tuple containing the `exteriors` and `interiors` lists. This allows you to separate the input polygons into exterior and interior polygons based on their orientation.

## 3.3 - Checking Points Within a Polygon

The function `points_inside_poly(points, polygon)` is a Python function that checks if a set of points are inside a given polygon. 

The function takes two parameters: `points` and `polygon`. `points` is expected to be an array of coordinates, where each coordinate is represented as a two-element list or tuple (for x and y values). `polygon` is expected to be a list of coordinates that define the vertices of the polygon.


In [25]:
def points_inside_poly(points, polygon):
    p = path.Path(polygon)
    return p.contains_points(points)

The function then returns the result of the `contains_points(points)` method called on the `Path` object `p`. This method checks if the points specified in the `points` parameter are contained within the polygon represented by the `Path` object. It returns a boolean array of length N (where N is the number of points), with each element indicating whether the corresponding point is inside the polygon (True) or not (False).

## 3.4 - Reversing the order of elements

The function `reverse_geometry(p)` is a Python function that reverses the order of elements in an array-like object along the vertical axis (up/down). 

The function takes one parameter, `p`, which is expected to be an array-like object. This could be a list, tuple, or a NumPy array. The function is designed to work with multi-dimensional arrays, but it can also handle 1-dimensional arrays.

Inside the function, the NumPy function `numpy.flipud(p)` is called with `p` as its argument. The `flipud` function stands for "flip up/down" and it reverses the order of the rows in a 2-D array. For a 1-D array, it simply reverses the order of the elements. 


In [26]:
def reverse_geometry(p):
    return numpy.flipud(p)

The result of the `flipud` function is then returned by the `reverse_geometry` function. 

## 3.5 - Geometry extraction
This Python function `extract_geometries` is used to extract geometrical data from a contour plot. The contour plot is passed as the first argument, while `data_min` and `data_max` are the minimum and maximum data values respectively.

In [27]:
def extract_geometries(contour, data_min, data_max):
  geoms = []
  vars = []

  # create list of tuples (geom, vmin, vmax)
  num_collections = len(contour.collections)
  for colli, coll in enumerate(contour.collections):

    # tricontourf generates extra collections for all geometries below
    # and above the given levels
    if colli == 0:
      vmin, vmax = data_min, contour.levels[colli]
    elif colli == num_collections-1:
      vmin, vmax = contour.levels[colli-1], data_max
    else:
      vmin, vmax = contour.levels[colli-1:colli+1]

    for p in coll.get_paths():
      p.simplify_threshold = 0.0
      polys = p.to_polygons()

      # don't append an empty polygon to the geometry or it will mess up
      # the indexing
      if polys:
        # remove all geometries with less than 3 points
        polys = [p for p in polys if p.shape[0] >= 3]

        # exteriors and interiors can be in any order (an exterior is not
        # necessarily followed by its interiors - test it!)
        exteriors, interiors = classify_polygons(polys)
        if len(interiors) > 0:
          interior_points = [pts[0] for pts in interiors]

        overall_inout = numpy.zeros((len(interiors),), dtype=bool)

        for exterior in exteriors:
          if len(interiors) > 0:
            inout = points_inside_poly(interior_points, exterior)
            overall_inout = numpy.logical_or(overall_inout, inout)

            my_interiors = [g for i, g in enumerate(interiors) if inout[i]]
            poly = Polygon(exterior, my_interiors)

          else:
            poly = Polygon(exterior)

          # clean-up polygons (remove intersections)
          if not poly.is_valid:
            poly = poly.buffer(0.0)

          geoms.append(poly)
          vars.append((vmin, vmax))

        # collect all interiors which do not belong to any of the exteriors
        outer_interiors = [interior for i, interior in enumerate(interiors) if not overall_inout[i]]
        for geom in outer_interiors:
          poly = Polygon(reverse_geometry(geom))

          # clean-up polygons (remove intersections)
          if not poly.is_valid:
            poly = poly.buffer(0.0)

          geoms.append(poly)
          vars.append((vmin, vmax))

  return geoms, vars

## 3.6 - Interactive plotting

This Python code is used to create an interactive plot using the `matplotlib` and `ipywidgets` libraries. The plot is based on a triangulation of data points, and the color of each triangle is determined by the corresponding data value.

In [28]:

def interactive_plot(Max_Level, Intervals):
    # matplotlib: triangulation
    triang = tri.Triangulation(x, y, triangles=elems)
    # check if data array is masked
    try:
        if data.mask.any():
            # -99999 entries in 'data' array are usually masked, mask all corresponding triangles
            point_mask_indices = numpy.where(data.mask)
            tri_mask = numpy.any(numpy.in1d(elems, point_mask_indices).reshape(-1, 3), axis=1)
            triang.set_mask(tri_mask)

    except AttributeError:
        # the "mask" attribute was not found, assume that no -99999 values are in the dataset
        print("No 'mask' attribute found in file '%s'" % infile)

    levels = numpy.linspace(0, Max_Level, num=Intervals)
    # print("levels: %s\n" % levels)

    # Matplotlib: Color plot
    c = plt.tricontourf(triang, data, levels=levels, cmap=plt.cm.jet, extend='both')
    c.cmap.set_under("#000066")
    c.cmap.set_over("#880066")
    plt.colorbar(c, ticks=levels)
    plt.show()

# Create sliders for maxlevel and Intervals
maxlevel_slider = widgets.IntSlider(min=1, max=10, step=1, value=2)#max data value
Intervals_slider = widgets.IntSlider(min=2, max=30, step=1, value=16)#number of colors

# Use interact function to automatically create UI
interact(interactive_plot, Max_Level=maxlevel_slider, Intervals=Intervals_slider)

interactive(children=(IntSlider(value=2, description='Max_Level', max=10, min=1), IntSlider(value=16, descript…

<function __main__.interactive_plot(Max_Level, Intervals)>